# Create file for DB upload

This script converts the data from `hex_hex` to a format that is better suited for the database import.

`hex_hex` contains the manual assignment of columnar neurons to their base column.

The result is stored in `results/exchange/ME_assigned_columns.csv`

In [ ]:
from pathlib import Path
import sys
from dotenv import load_dotenv, find_dotenv
load_dotenv()
PROJECT_ROOT = Path(find_dotenv()).parent
sys.path.append(str(PROJECT_ROOT.joinpath("src")))
print(f"Project root directory: {PROJECT_ROOT}")

from utils import olc_client
c = olc_client.connect(verbose=True)

In [ ]:
from utils.hex_hex import get_hex_df

In [ ]:
all_columns = get_hex_df()

long_col = all_columns\
    .melt(id_vars=['hex1_id', 'hex2_id'], var_name='neuron_type', value_name='bodyId')\
    .dropna()

upload_format = long_col[~long_col['bodyId'].duplicated()]\
    .set_index(['bodyId', 'neuron_type'])\
    .sort_values(by=['hex1_id', 'hex2_id'])\
    .rename({'hex1_id': 'assigned_hex1', 'hex2_id': 'assigned_hex2'}, axis=1)


In [ ]:
data_path = Path(find_dotenv()).parent / 'results' / 'exchange'
data_path.mkdir(exist_ok=True)
upload_format.to_csv(data_path / 'ME_assigned_columns.csv')